In [2]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as tfhub
import sqlite3
from sqlite3 import Error

pd.set_option('display.max_colwidth', -1)

In [3]:
conn = sqlite3.connect('wine_data.sqlite')
c = conn.cursor()

In [4]:
wine_df = pd.read_sql('Select * from wine_data', conn)

In [5]:
wine_df.head()

,index,country,description,rating,price,province,title,variety,winery,color
0,0,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red
1,1,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white
2,2,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,white
3,3,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,red
4,4,France,"This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.",87,24.0,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,white


In [12]:
embed = tfhub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

In [9]:
#download the model to local so it can be used again and again
#!mkdir ../wine_model/module_useT
# Download the module, and uncompress it to the destination folder. 
#!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ../wine_model/module_useT

The syntax of the command is incorrect.


In [7]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  wine_embeddings = session.run(embed(list(wine_df.description)))
print(wine_embeddings.shape)

(100228, 512)


In [ ]:
def recommend_engine(query, embedding_table = wine_embeddings):
    # Embed user query
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embedding = session.run(embed([query]))

    # Calculate similarity with all reviews
    similarity_score = np.dot(embedding, embedding_table.T)
    
    # Recommend
    recommendations = wine_df.copy()
    recommendations['recommendation'] = similarity_score.T
    recommendations = recommendations.sort_values('recommendation', ascending=False)
    recommendations = recommendations[['variety', 'title', 'price', 'description', 'recommendation', 'score']]

    return recommendations

In [ ]:
query = "fruity, rich, easy to drink, sweet"

recommendation = recommend_engine(query)
print(query)

recommendation.head(5).T